In [91]:
import os
import pandas as pd
import numpy as np

import pmdarima as pm
import keras.backend as K
from keras.layers import LSTM, Dense, Input, Bidirectional
from keras.optimizers import Adagrad
from keras.losses import mean_squared_logarithmic_error
from keras.models import load_model, Model
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression


In [92]:
DATA_DIR = 'data'
OUTPUT_FILE = 'submission.csv'
TRAIN_SEQ_SIZE = 62
TEST_SEQ_SIZE = 43

def load_initial_data():
    train_path = os.path.join(DATA_DIR, 'train.csv')
    test_path = os.path.join(DATA_DIR, 'test.csv')
    train_data, test_data = pd.read_csv(train_path), pd.read_csv(test_path)
    return train_data, test_data

def load_extended_data():
    train_path = os.path.join(DATA_DIR, 'extended_train.csv')
    test_path = os.path.join(DATA_DIR, 'extended_test.csv')
    train_data, test_data = pd.read_csv(train_path), pd.read_csv(test_path)
    return train_data, test_data

In [93]:
def extract_days_from_first_infection(data: pd.DataFrame, write_file=False):
    result = list()
    prev_region, prev_country = None, None
    current_count = 0
    found_case = False
    for value in data.values:
        if prev_country is None:
            prev_region, prev_country = value[1], value[2]
        if value[1] is prev_region and value[2] == prev_country:
            if value[14] == 0:
                result.append(current_count)
            elif value[14] != 0 and not found_case:
                found_case = True
                result.append(current_count)
            elif value[14] != 0 and found_case:
                current_count += 1
                result.append(current_count)
        else:
            found_case = False
            current_count = 0
            if value[14] == 0:
                result.append(current_count)
            elif value[14] != 0:
                found_case = True
                result.append(current_count)
        prev_region, prev_country = value[1], value[2]
    data['Days since first infection'] = result
    if write_file:
        data.to_csv('added_days_train.csv', index=False)
    

def add_days_from_first_infection_test(train_data: pd.DataFrame, test_data: pd.DataFrame, write_file=False):
    max_counts = dict()
    result = list()
    for value in train_data.values:
        max_counts[str(value[1]) + str(value[2])] = value[-1]
    previous_key = None
    current_count = 0
    for value in test_data.values:
        key = str(value[1]) + str(value[2])
        if previous_key is None or key != previous_key:
            current_count = max_counts[key] + 1
        else:
            current_count += 1
        result.append(current_count)
        previous_key = key
    test_data['Days since first infection'] = result
    if write_file:
        test_data.to_csv('added_days_test.csv', index=False)

In [94]:
def replace_missing_extra_values_with_mean(data, write_file=False):
    data = data.mask(data == 0).fillna(data.mean())
    if write_file:
        data.to_excel('extra_features_improved.xlsx', index=False)

def merge_with_extra(train_df: pd.DataFrame, test_df: pd.DataFrame, extra_df: pd.DataFrame, write_file=False):
    train_df = train_df.merge(extra_df, how='left', on='Country_Region')
    test_df = test_df.merge(extra_df, how='left', on='Country_Region')
    if write_file:
        train_df.to_csv('extended_train_merged.csv', index=False)
        test_df.to_csv('extended_test_merged.csv', index=False)
    

In [95]:
def root_mean_squared_log_error(y_true, y_predicted):
    return K.sqrt(mean_squared_logarithmic_error(y_true, y_predicted))


def build_model(train_features, train_labels, build_version=1, load_path=None):
    if load_path:
        model = load_model(load_path)
    else:
        if build_version == 1:
            inputs = Input(shape=train_features[0].shape)
            lstm_1 = LSTM(units=16, activation='softsign', return_sequences=True)(inputs)
            lstm_2 = LSTM(units=8, activation='softsign', return_sequences=True)(lstm_1)
            dense = Dense(4, activation='relu')(lstm_2)
            output = Dense(2)(dense)
            model = Model(inputs=inputs, outputs=output)
            model.compile(optimizer=Adagrad(), loss='mse', metrics=['acc'])
            model.summary()
            model.fit(train_features, train_labels, batch_size=64, epochs=10)
            model.save('model_v1.h5')
        elif build_version == 2:
            inputs = Input(shape=train_features[0].shape)
            lstm_1 = Bidirectional(LSTM(units=32, activation='softsign', return_sequences=True))(inputs)
            lstm_2 = Bidirectional(LSTM(units=32, activation='softsign', return_sequences=True))(lstm_1)
            dense = Dense(4, activation='relu')(lstm_2)
            output = Dense(2)(dense)
            model = Model(inputs=inputs, outputs=output)
            model.compile(optimizer=Adagrad(), loss='mse', metrics=['acc'])
            model.summary()
            model.fit(train_features, train_labels, batch_size=64, epochs=10)
            model.save('model_v2.h5')
        elif build_version == 3:
            inputs = Input(shape=train_features[0].shape)
            lstm_1 = Bidirectional(LSTM(units=32, activation='softsign', return_sequences=True))(inputs)
            lstm_2 = Bidirectional(LSTM(units=32, activation='softsign', return_sequences=True))(lstm_1)
            dense = Dense(4, activation='relu')(lstm_2)
            output_1 = Dense(1)(dense)
            output_2 = Dense(1)(dense)
            model = Model(inputs=inputs, outputs=[output_1, output_2])
            model.compile(optimizer=Adagrad(), 
                          loss=[root_mean_squared_log_error, root_mean_squared_log_error],
                          metrics=['acc'])
            model.summary()
            model.fit(train_features, train_labels, batch_size=64, epochs=10)
            model.save('model_v3.h5')
        else:
            inputs = Input(shape=train_features[0].shape)
            lstm_1 = LSTM(units=4, activation='softsign', return_sequences=True)(inputs)
            output_1 = Dense(1)(lstm_1)
            output_2 = Dense(1)(lstm_1)
            model = Model(inputs=inputs, outputs=[output_1, output_2])
            model.compile(optimizer=Adagrad(), 
                          loss=[root_mean_squared_log_error, root_mean_squared_log_error],
                          metrics=['acc'])
            model.summary()
            model.fit(train_features, train_labels, batch_size=64, epochs=10)
            model.save('model_v4.h5')
    return model

In [96]:
def get_features(df: pd.DataFrame, build_version=1, seq_len=TRAIN_SEQ_SIZE, for_train=True):
    df['Province_State'] = df['Province_State'].fillna('<placeholder>')
    results = [group for _, group in df.groupby(['Country_Region', 'Province_State'])]
    groups = np.stack(results)
    if for_train:
        feature_columns = [i for i in range(4, 20) if i not in [14, 15]]
        label_columns = [14, 15]
    else:
        feature_columns = [i for i in range(4, 18)]
        label_columns = None
    features = groups[:, :, feature_columns]
    features = pad_sequences(features, seq_len, padding='post', truncating='post', dtype='float32')
    if label_columns is not None:
        if build_version >= 3:
            labels_1 = groups[:, :, label_columns[0]]
            labels_1 = pad_sequences(labels_1, seq_len, padding='post', truncating='post', dtype='float32')
            labels_2 = groups[:, :, label_columns[1]]
            labels_2 = pad_sequences(labels_2, seq_len, padding='post', truncating='post', dtype='float32')
            return features, [labels_1, labels_2]
        else:
            labels = groups[:, :, label_columns]
            labels = pad_sequences(labels, seq_len, padding='post', truncating='post', dtype='float32')
            return features, labels
    return features, None


def get_regression_features(df:pd.DataFrame, for_train=True):
    df['Province_State'] = df['Province_State'].fillna('<placeholder>')
    results = [group for _, group in df.groupby(['Country_Region', 'Province_State'])]
    groups = np.stack(results)
    if for_train:
        feature_columns = [i for i in range(4, 20) if i not in [14, 15]]
        label_columns = [14, 15]
    else:
        feature_columns = [i for i in range(4, 18)]
        label_columns = None
    features = groups[:, :, feature_columns]
    if label_columns is not None:
        labels = groups[:, :, label_columns]
        return features, labels
    return features, None


def get_arima_features(df:pd.DataFrame, for_train=True):
    df['Province_State'] = df['Province_State'].fillna('<placeholder>')
    results = [group for _, group in df.groupby(['Country_Region', 'Province_State'])]
    groups = np.stack(results)
    if for_train:
        feature_columns = [i for i in range(4, 20) if i not in [14, 15]]
        label_columns = [14, 15]
    else:
        feature_columns = [i for i in range(4, 18)]
        label_columns = None
    features = groups[:, :, feature_columns]
    if label_columns is not None:
        labels_1 = groups[:, :, label_columns[0]]
        labels_2 = groups[:, :, label_columns[1]]
        return features, [labels_1, labels_2]
    return features, None

In [97]:
def create_nn_output_file(predictions, build_version=1):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        if build_version < 3:
            for pred in predictions:
                for i in range(TEST_SEQ_SIZE):
                    f.write(f'{str(count)},{str(int(pred[i][0]))},{str(int(pred[i][1]))}\n')
                    count += 1
        else:
            for first_pred, second_pred in zip(*predictions):
                for i in range(TEST_SEQ_SIZE):
                    f.write(f'{str(count)},{str(first_pred[i][0])},{str(second_pred[i][0])}\n')
                    count += 1


def create_regression_output_file(predictions):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        for country in predictions:
            for day in country:
                f.write(f'{str(count)},{str(int(day[0]))},{str(int(day[1]))}\n')
                count += 1


def create_arima_output_file(cases_predictions, fatalities_predictions):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        for prediction in zip(cases_predictions, fatalities_predictions):
            f.write(f'{str(count)},{str(int(prediction[0]))},{str(int(prediction[1]))}\n')
            count += 1


def normalize_input(train, test):
    # reshape for MinMaxScaler
    train_dims, test_dims = train.shape, test.shape
    train_features = train.reshape(train_dims[0], train_dims[1] * train_dims[2])
    test_features = test.reshape(test_dims[0], test_dims[1] * test_dims[2])
    
    normalizer = MinMaxScaler()
    normalizer = normalizer.fit(train_features)
    train_scaled = normalizer.transform(train_features)
    test_scaled = normalizer.transform(test_features)
    
    # reshape scaled back to 3 dimensions
    train_scaled = train_scaled.reshape(train_dims)
    test_scaled = test_scaled.reshape(test_dims)
    return train_scaled, test_scaled

In [98]:
def neural_main(build_version=3, use_normalization=False):
    train, test = load_extended_data()

    # 62 is the sequence length for train data and 43 for the testing data
    train_features, train_labels = get_features(train, build_version)
    test_features, _ = get_features(test, build_version, for_train=False)
    if use_normalization:
        train_features, test_features = normalize_input(train_features, test_features)
    if build_version >= 3:
        train_labels_1 =  train_labels[0][:, :, np.newaxis]
        train_labels_2 =  train_labels[1][:, :, np.newaxis]
        model = build_model(train_features, [train_labels_1, train_labels_2], build_version=build_version)
    else:
        model = build_model(train_features, train_labels, build_version=build_version)
    predictions = model.predict(test_features)
    create_nn_output_file(predictions, build_version=build_version)


In [99]:
def regression_main(use_exponential=False):
    train, test = load_extended_data()
    train_features, train_labels = get_regression_features(train)
    test_features, _ = get_regression_features(test, for_train=False)
    
    # for each country fit a separate model
    predictions = list()
    for country in range(train_features.shape[0]):
        clf = LinearRegression()
        if not use_exponential:
            clf.fit(train_features[country], train_labels[country])
            prediction = clf.predict(test_features[country])
        else:
            # TODO: Needs weights for a better prediction
            y_shape = train_labels[country].shape
            
            log_labels = [np.log(train_labels[country][i][j]) for i in range(y_shape[0]) 
                          for j in range(y_shape[1])]
            log_labels = np.array(log_labels)
            log_labels = np.nan_to_num(log_labels)
            # scipy Bug fix below!!!
            log_labels[log_labels < 0.00000001] = 0.00000001
            log_labels = np.reshape(log_labels, y_shape)

            clf.fit(train_features[country], log_labels)
            prediction = np.exp(clf.predict(test_features[country]))
        predictions.append(prediction)
    create_regression_output_file(predictions)


def arima_main():
    train, test = load_extended_data()
    train_features, train_labels = get_arima_features(train)
    test_features, _ = get_arima_features(test, for_train=False)
    
    # for each country fit a separate model
    cases_predictions = list()
    fatalities_predictions = list()
    for country in range(train_features.shape[0]):
        print(country)
        # Endogenous -> train_labels (ConfirmedCases + Fatalities)
        # Exogenous -> train_features (Everything we've collected)
        cases_model = pm.arima.AutoARIMA(seasonal=False, suppress_warnings=True)
        cases_model.fit(train_labels[0][country], exogenous=train_features[country])
        cases_prediction = cases_model.predict(n_periods=test_features.shape[1], 
                                               exogenous=test_features[country])
        cases_predictions.extend(cases_prediction)
        fatalities_model = pm.arima.AutoARIMA(seasonal=False, suppress_warnings=True)
        fatalities_model.fit(train_labels[1][country], exogenous=train_features[country])
        fatalities_prediction = fatalities_model.predict(n_periods=test_features.shape[1], 
                                                         exogenous=test_features[country])
        fatalities_predictions.extend(fatalities_prediction)
    create_arima_output_file(cases_predictions, fatalities_predictions)


# neural_main(build_version=4)
# regression_main(use_exponential=False)
arima_main()
# train, test = load_extended_data()
# extract_days_from_first_infection(train, True)
# add_days_from_first_infection_test(train, test, True)
# extra = pd.read_excel('data/extra_features.xlsx')
# merge_with_extra(train, test, extra, True)

0
1
2
3
4


c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1886: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1893: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1893: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\pmdarima\arima\auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


5


c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\pmdarima\arima\auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


6


...

290
291


c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\pmdarima\arima\auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


292


c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1893: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
c:\users\serban\appdata\local\programs\python\python37\lib\site-packages\pmdarima\arima\auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


293
